### Import libraries

In [55]:
# Import necessary libraries
import pulp
import pandas as pd

### Customize

In [56]:
# Provide customizable paraneters 
products       = ["Chair", "Table", "Bookshelf"]
profit_per_unit= [228, 306, 482]
cost_per_unit  = [272, 544, 668]
min_demand     = [10000, 7000, 6000]
max_demand     = [17000, 13000, 11000]
labour_hours   = [1, 2, 1.5]
wood_units     = [20, 40, 50]

total_labour_available = 45000
total_wood_available = 1300000
minimum_profit     = 8000000

my_output="0305 optimal_production_plan.csv"
my_currency="$"

### Define LP problem

In [57]:
# Specify parameters for LP
n = len(products)

#  Define LP model 

# Maximize profit (Comment out if not using this)
model = pulp.LpProblem("Profit_Maximization", pulp.LpMaximize)

# Minimize cost (Comment out if not using this)
#model = pulp.LpProblem("Cost_Minimization", pulp.LpMinimize)
 
# Decision variables
x = [pulp.LpVariable(f"x_{i}", lowBound=0, cat="Integer") for i in range(n)]

# Objective function: Maximize total profit, comment out for cost minimization
model += pulp.lpSum([profit_per_unit[i] * x[i] for i in range(n)])

# Objective function: Minimize total cost, comment out for profit maximization
#model += pulp.lpSum([cost_per_unit[i] * x[i] for i in range(n)])

# Common constraints
model += pulp.lpSum([labour_hours[i] * x[i] for i in range(n)]) <= total_labour_available
model += pulp.lpSum([wood_units[i] * x[i] for i in range(n)]) <= total_wood_available

# Constraint for cost minimization 
#model += pulp.lpSum([profit_per_unit[i] * x[i] for i in range(n)]) >= minimum_profit

#Demand constraint
for i in range(n):
    model += x[i] >= min_demand[i]
    model += x[i] <= max_demand[i]

### Solve LP problem

In [58]:
# Solve the Problem
model.solve()

1

### Prepare summary and save results

In [59]:
# Print Results 
results = []
total_profit = 0
total_cost = 0
total_labour_used = 0
total_wood_used = 0

for i in range(n):
    qty = int(x[i].varValue)
    total_line_cost = qty * cost_per_unit[i]
    profit_value = qty * profit_per_unit[i]
    labour_used = qty * labour_hours[i]
    wood_used = qty * wood_units[i]

    total_profit += profit_value
    total_cost += total_line_cost
    total_labour_used += labour_used
    total_wood_used += wood_used

    results.append({
        "Product": products[i],
        "ProfitPerUnit": profit_per_unit[i],
        "MinDemand": min_demand[i],
        "MaxDemand": max_demand[i],
        "LabourPerUnit": labour_hours[i],
        "WoodPerUnit": wood_units[i],
        "QtyProduced": qty,
        "TotalCost" :total_line_cost,
        "Profit": profit_value,
        "LabourUsed": labour_used,
        "WoodUsed": wood_used
    })


In [60]:
# Add summary row
results.append({
    "Product": "TOTAL",
    "ProfitPerUnit": "",
    "MinDemand": "",
    "MaxDemand": "",
    "LabourPerUnit": "",
    "WoodPerUnit": "",
    "QtyProduced": "",
    "TotalCost" :total_cost,
    "Profit": total_profit,
    "LabourUsed": total_labour_used,
    "WoodUsed": total_wood_used
})

# Print results
print(f"Total profit at optimal solution: {my_currency} {total_profit}")
print(f"Total cost at optimal solution: {my_currency} {total_cost}")
print(f"Total Labour Used: {total_labour_used} out of available {total_labour_available}")
print(f"Total Wood Used: {total_wood_used} out of available {total_wood_available}")
for r in results[:-1]:  
    print(f"→ {r['Product']}: Produce {r['QtyProduced']} units")

# Save to CSV ===
df = pd.DataFrame(results)
df.to_csv(my_output, index=False)
print(f"Results saved to {my_output}")


Total profit at optimal solution: $ 10750000
Total cost at optimal solution: $ 15100000
Total Labour Used: 45000.0 out of available 45000
Total Wood Used: 1120000 out of available 1300000
→ Chair: Produce 14500 units
→ Table: Produce 7000 units
→ Bookshelf: Produce 11000 units
Results saved to 0305 optimal_production_plan.csv
